In [177]:
# !pip install vaderSentiment
# !pip install imblearn

In [1]:
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [3]:
df = pd.read_csv('https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Health_Personal_Care_v1_00.tsv.gz', sep='\t', error_bad_lines=False)
df

b'Skipping line 10317: expected 15 fields, saw 22\nSkipping line 40632: expected 15 fields, saw 22\nSkipping line 49643: expected 15 fields, saw 22\nSkipping line 54096: expected 15 fields, saw 22\nSkipping line 63958: expected 15 fields, saw 22\nSkipping line 64322: expected 15 fields, saw 22\n'
b'Skipping line 112718: expected 15 fields, saw 22\n'
b'Skipping line 156739: expected 15 fields, saw 22\nSkipping line 158865: expected 15 fields, saw 22\nSkipping line 192178: expected 15 fields, saw 22\n'
b'Skipping line 228364: expected 15 fields, saw 22\nSkipping line 228763: expected 15 fields, saw 22\nSkipping line 233558: expected 15 fields, saw 22\nSkipping line 234939: expected 15 fields, saw 22\nSkipping line 247329: expected 15 fields, saw 22\nSkipping line 250514: expected 15 fields, saw 22\nSkipping line 259994: expected 15 fields, saw 22\n'
b'Skipping line 278327: expected 15 fields, saw 22\nSkipping line 287267: expected 15 fields, saw 22\nSkipping line 292441: expected 15 fiel

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,650634,R3EQSTM9PWRAL,B0091LBZSU,578484426,Demograss Capsules Diet Pills Lose Weight,Health & Personal Care,3,0.0,0.0,N,Y,Three Stars,Only came with 30 pills I wish it would've cam...,2015-08-31
1,US,19827510,RBWPRK17XKIXD,B00PWW3LQ6,456433146,Viva Labs #1 Premium Himalayan Organic Goji Be...,Health & Personal Care,5,0.0,0.0,N,Y,A good portable snack,Really yummy and healthy. A good portable snack.,2015-08-31
2,US,1520474,RRSLOAF273XFC,B00DKEWA92,460764511,Whip-it Cleaner,Health & Personal Care,5,1.0,2.0,N,Y,You need this,You cannot go wrong with this stuff. I use it ...,2015-08-31
3,US,23905905,R3S8W9Q6SWIT8O,B0015R3A7M,135102038,Optimum Nutrition Opti-Men Daily Multivitamin ...,Health & Personal Care,4,0.0,0.0,N,Y,this is a good vitamin for men,This is a good daily vitamin for men.,2015-08-31
4,US,28215779,R3QQ6NSLRVBFJC,B006B8U8BG,200666829,Wet-Stop3 Bedwetting Enuresis Alarm with Loud ...,Health & Personal Care,4,0.0,0.0,N,Y,progress is definitely happening!!!!,We have been using this for almost a month and...,2015-08-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5312728,US,52339912,R10ITC6DGYFD3J,B00000JHQ7,128840518,Duracell MN1500B8 AA (8-Pack),Health & Personal Care,1,11.0,46.0,N,N,"A disappointment, to say the least.",I expected so much and yet recieved so little ...,1999-10-21
5312729,US,51112985,R16BFEO1ITQUP9,B00000JHQ6,131822021,Duracell Coppertop Alkaline Batteries,Health & Personal Care,4,3.0,4.0,N,N,Stuff doesn't work without them :),"Duracell is a good brand, and I recomend them ...",1999-09-24
5312730,US,51859493,RJGC0F0GOD6QN,B00000JHQ7,128840518,Duracell MN1500B8 AA (8-Pack),Health & Personal Care,5,8.0,13.0,N,N,The best portable source of power I've found y...,These things are absolutely marvelous! Seriou...,1999-08-26
5312731,US,52044246,R2RTO9RPW3E7GE,B00000JHQ5,222843861,"Duracell MN1500B2 AA, (2-Pack)",Health & Personal Care,5,1.0,8.0,N,N,The Copper-Top Still Rocks to the Top!,"&quot;The Batteries I use are called Duracell,...",1999-08-07


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5312733 entries, 0 to 5312732
Data columns (total 15 columns):
marketplace          object
customer_id          int64
review_id            object
product_id           object
product_parent       int64
product_title        object
product_category     object
star_rating          object
helpful_votes        float64
total_votes          float64
vine                 object
verified_purchase    object
review_headline      object
review_body          object
review_date          object
dtypes: float64(2), int64(2), object(11)
memory usage: 608.0+ MB


In [5]:
analyzer = SentimentIntensityAnalyzer()

In [6]:
def sentiment_analyzer_scores(sentence, verbose=False):
    score = analyzer.polarity_scores(sentence)
    if verbose: print("{}".format(str(score)))
    
    return score

    # compound_score = score['compound']

    # if compound_score >= .05: return 1
    # if compound_score > -.05: return 0

    # return -1
    
def sentiment_analyzer_scores_neg(sentence):
    return sentiment_analyzer_scores(sentence)['neg']

def sentiment_analyzer_scores_neu(sentence):
    return sentiment_analyzer_scores(sentence)['neu']

def sentiment_analyzer_scores_pos(sentence):
    return sentiment_analyzer_scores(sentence)['pos']

def sentiment_analyzer_scores_compound(sentence):
    return sentiment_analyzer_scores(sentence)['compound']

In [7]:
df_test = df[:20000]
df_test['star_rating'] = df_test['star_rating'].astype(int)
df_test['review_body'] = df_test['review_body'].fillna('.')
df_test['review_fulltext'] = df_test['review_headline'] + '. ' + df_test['review_body']

df_test['neg'] = df_test['review_fulltext'].apply(sentiment_analyzer_scores_neg)
df_test['neu'] = df_test['review_fulltext'].apply(sentiment_analyzer_scores_neu)
df_test['pos'] = df_test['review_fulltext'].apply(sentiment_analyzer_scores_pos)
df_test['compound'] = df_test['review_fulltext'].apply(sentiment_analyzer_scores_compound)

df_test.info()

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = val

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 20 columns):
marketplace          20000 non-null object
customer_id          20000 non-null int64
review_id            20000 non-null object
product_id           20000 non-null object
product_parent       20000 non-null int64
product_title        20000 non-null object
product_category     20000 non-null object
star_rating          20000 non-null int64
helpful_votes        20000 non-null float64
total_votes          20000 non-null float64
vine                 20000 non-null object
verified_purchase    20000 non-null object
review_headline      20000 non-null object
review_body          20000 non-null object
review_date          20000 non-null object
review_fulltext      20000 non-null object
neg                  20000 non-null float64
neu                  20000 non-null float64
pos                  20000 non-null float64
compound             20000 non-null float64
dtypes: float64(6), int64(3)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [8]:
df_test['star_rating'].value_counts()

5    13139
4     2681
1     1832
3     1401
2      947
Name: star_rating, dtype: int64

In [9]:
df_numer = df_test.select_dtypes(include='number')
df_numer.info()
df_numer

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 9 columns):
customer_id       20000 non-null int64
product_parent    20000 non-null int64
star_rating       20000 non-null int64
helpful_votes     20000 non-null float64
total_votes       20000 non-null float64
neg               20000 non-null float64
neu               20000 non-null float64
pos               20000 non-null float64
compound          20000 non-null float64
dtypes: float64(6), int64(3)
memory usage: 1.4 MB


,customer_id,product_parent,star_rating,helpful_votes,total_votes,neg,neu,pos,compound
0,650634,578484426,3,0.0,0.0,0.000,0.838,0.162,0.4019
1,19827510,456433146,5,0.0,0.0,0.000,0.325,0.675,0.9092
2,1520474,460764511,5,1.0,2.0,0.000,0.876,0.124,0.7873
3,23905905,135102038,4,0.0,0.0,0.000,0.469,0.531,0.8481
4,28215779,200666829,4,0.0,0.0,0.067,0.850,0.083,-0.3287
...,...,...,...,...,...,...,...,...,...
19995,30099085,396679839,5,0.0,0.0,0.000,0.676,0.324,0.9245
19996,52273405,798034451,1,0.0,0.0,0.113,0.833,0.054,-0.6652
19997,43059918,973721042,4,0.0,0.0,0.000,0.613,0.387,0.9225
19998,18111500,332910265,5,0.0,1.0,0.030,0.705,0.264,0.9877


In [10]:
X = df_test.loc[:, ['helpful_votes', 'total_votes', 'neg', 'neu', 'pos', 'compound']]
y = df_test.loc[:, 'star_rating']

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)

In [12]:
y_train.value_counts()

5    9927
4    2006
1    1361
3    1024
2     682
Name: star_rating, dtype: int64

In [13]:
smote = SMOTE()
X_train_resampled, y_train_resampled = smote.fit_sample(X_train, y_train) 
y_train_resampled.value_counts()

5    9927
4    9927
3    9927
2    9927
1    9927
Name: star_rating, dtype: int64

In [ ]:
from sklearn.cluster import KMeans

num_clusters = np.arange(2, 50)

# Set number of clusters at initialization time
for n in num_clusters:
    k_means = KMeans(n_clusters=n) 

    # Run the clustering algorithm
    k_means.fit(df_numer) 

    # Generate cluster index values for each row
    cluster_assignments = k_means.predict(df_numer)
    
    print(n, calinski_harabasz_score(df_numer, cluster_assignments))

In [ ]:
cluster_assignments

In [ ]:
k_means.labels_

In [ ]:
# This code builds on the previous example
from sklearn.metrics import calinski_harabasz_score

# Note that we could also pass in k_means.labels_ instead of cluster_assignments
print(calinski_harabasz_score(df_numer, cluster_assignments))